In [1]:
#file1=['../tmp/kgtk_roget_synonyms.tsv', '../tmp/kgtk_roget_antonyms.tsv']
#file1=['../tmp/kgtk_conceptnet.tsv']

files=['../tmp/kgtk_wordnet.tsv', '../tmp/kgtk_wikidata.tsv', '../tmp/kgtk_conceptnet.tsv']

In [2]:
from tqdm import tqdm
import json

In [3]:
dim_file='../consolidation/dimensions.json'

In [4]:
with open(dim_file, 'r') as f:
    dim_mapping=json.load(f)

In [5]:
dimensions=set(dim_mapping.values())

In [6]:
len(dimensions)

12

In [7]:
dimensions

{'creation',
 'desire',
 'distinctness',
 'lexical',
 'part-whole',
 'quality',
 'rel-other',
 'similarity',
 'spatial',
 'taxonomic',
 'temporal',
 'utility'}

In [8]:
def get_all_labels(l):
    if '|' not in l:
        return [l]
    else:
        return [l2 for l2 in l.split('|') if l2!='']

In [13]:
def get_triples(a_file, abstract_rel=False, dim_mapping={}, dim_filter=None):
    triples=set()
    with open(a_file, 'r') as f1:
        header=next(f1)
        for line in tqdm(f1, total=3500000):
            data=line.split('\t')
            rel_label=data[1]
            if abstract_rel:
                if rel_label not in dim_mapping.keys(): continue
                if dim_filter:
                    rel_label=dim_mapping[rel_label]
                    if rel_label!=dim_filter: continue
            for n1_label in get_all_labels(data[3]):
                for n2_label in get_all_labels(data[4]):
                    if n1_label=='' or n2_label=='': print(data)
                    triple=(n1_label, rel_label, n2_label)
                    triples.add(triple)
    return triples

## Compute overlaps between each source pair

In [14]:
def file_to_source(f):
    return f.split('/')[-1].replace('kgtk_', '').replace('.tsv', '')

In [15]:
for file1 in files:
    for file2 in files:
        if file1<file2:

            f1_triples=get_triples(file1)
            f2_triples=get_triples(file2)
            both_triples=set(f1_triples) & set(f2_triples)

            only_src2=len(set(f2_triples)-set(f1_triples))
            pct_only_src2=only_src2/(only_src2+len(both_triples))

            only_src1=len(set(f1_triples)-set(f2_triples))
            pct_only_src1=only_src1/(only_src1+len(both_triples))

            f1=file_to_source(file1)
            f2=file_to_source(file2)

            print(f1, '-', f2, 'BOTH:', len(both_triples), 'ONLY S1:', only_src1, 'ONLY S2:', only_src2)

  3%|▎         | 105560/3500000 [00:00<00:06, 509994.77it/s]

wikidata - wordnet BOTH: 1613 ONLY S1: 98246 ONLY S2: 419103


  3%|▎         | 104597/3500000 [00:00<00:06, 506765.46it/s]

conceptnet - wordnet BOTH: 66102 ONLY S1: 3257219 ONLY S2: 354614


  3%|▎         | 101771/3500000 [00:00<00:05, 591695.38it/s]


conceptnet - wikidata BOTH: 2386 ONLY S1: 3320935 ONLY S2: 97473


## Compute mappings (with dimensions instead of relations)

In [16]:
for file1 in files:
    for file2 in files:
        if file1<file2:

            f1_triples=get_triples(file1, True, dim_mapping)
            f2_triples=get_triples(file2, True, dim_mapping)
            both_triples=set(f1_triples) & set(f2_triples)

            only_src2=len(set(f2_triples)-set(f1_triples))
            pct_only_src2=only_src2/(only_src2+len(both_triples))

            only_src1=len(set(f1_triples)-set(f2_triples))
            pct_only_src1=only_src1/(only_src1+len(both_triples))

            f1=file_to_source(file1)
            f2=file_to_source(file2)

            print(f1, '-', f2, 'BOTH:', len(both_triples), 'ONLY S1:', only_src1, 'ONLY S2:', only_src2)

  3%|▎         | 100923/3500000 [00:00<00:06, 491480.87it/s]

wikidata - wordnet BOTH: 1613 ONLY S1: 98246 ONLY S2: 419103


  3%|▎         | 98437/3500000 [00:00<00:07, 475514.25it/s]

conceptnet - wordnet BOTH: 66102 ONLY S1: 3244955 ONLY S2: 354614


  3%|▎         | 101771/3500000 [00:00<00:06, 546796.74it/s]


conceptnet - wikidata BOTH: 2386 ONLY S1: 3308671 ONLY S2: 97473


### Compute mappings per dimension

In [18]:
for file1 in files:
    for file2 in files:
        if file1<file2:

            for d in dimensions:

                f1_triples=get_triples(file1, True, dim_mapping, d)
                f2_triples=get_triples(file2, True, dim_mapping, d)
                both_triples=set(f1_triples) & set(f2_triples)
            
                try:

                    only_src2=len(set(f2_triples)-set(f1_triples))
                    pct_only_src2=only_src2/(only_src2+len(both_triples))

                    only_src1=len(set(f1_triples)-set(f2_triples))
                    pct_only_src1=only_src1/(only_src1+len(both_triples))

                    f1=file_to_source(file1)
                    f2=file_to_source(file2)

                    print(f1, '-', f2, 'DIMENSION', d, 'BOTH:', len(both_triples), 'ONLY S1:', only_src1, 'ONLY S2:', only_src2)
                except:
                    continue

  0%|          | 0/3500000 [00:00<?, ?it/s]

wikidata - wordnet DIMENSION part-whole BOTH: 82 ONLY S1: 8121 ONLY S2: 100839


  0%|          | 0/3500000 [00:00<?, ?it/s]

wikidata - wordnet DIMENSION taxonomic BOTH: 1533 ONLY S1: 69884 ONLY S2: 318236


  2%|▏         | 85499/3500000 [00:00<00:03, 854980.62it/s]

...

conceptnet - wikidata DIMENSION creation BOTH: 0 ONLY S1: 262 ONLY S2: 187


  5%|▌         | 182395/3500000 [00:00<00:03, 845470.30it/s]

conceptnet - wikidata DIMENSION taxonomic BOTH: 1888 ONLY S1: 232819 ONLY S2: 69529


  3%|▎         | 101771/3500000 [00:00<00:03, 885804.40it/s]

conceptnet - wikidata DIMENSION temporal BOTH: 1 ONLY S1: 69016 ONLY S2: 2600


## Thoughts/discussion

* also with WordNet based nodes?


'dog' '/r/IsA' 'mammal'
dog.n.1 '/r/IsA' mammal.n.01

(There is Kypher query for this)

Discussion in the paper: what does a 'failure of overlap' mean? Is it due to the label missing or due to the edge missing?